In [1]:
import pandas as pd
import numpy as np
import glob
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('default')
  
# %matplotlib inline: only draw static
# images in the notebookº
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Carga de información

In [2]:
laliga2122 = pd.read_excel('datos_liga/LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('datos_liga/LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('datos_liga/LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('datos_liga/LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('datos_liga/LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('datos_liga/LaLiga16 17.xlsx', )
seg2122 =  pd.read_excel('datos_liga/LaLiga123 21 22.xlsx', )
seg2021 =  pd.read_excel('datos_liga/LaLiga123 20 21.xlsx', )
seg1920 =  pd.read_excel('datos_liga/LaLiga123 19 20.xlsx', )
seg1819 =  pd.read_excel('datos_liga/LaLiga123 18 19.xlsx', )
seg1718 =  pd.read_excel('datos_liga/LaLiga123 17 18.xlsx', )
seg1617 =  pd.read_excel('datos_liga/LaLiga123 16 17.xlsx', )


## Creando dataframes

In [3]:
laliga = laliga2122.dropna(how='all').append(laliga2021.dropna(how='all')
).append(laliga1920.dropna(how='all')).append(laliga1819.dropna(how='all')
).append(laliga1718.dropna(how='all')).append(laliga1617.dropna(how='all')).loc[:,
['Wk','Date','Local','Score','Visitante', 'Attendance']]

segunda = seg2122.dropna(how='all').append(seg2021.dropna(how='all')
).append(seg1920.dropna(how='all')).append(seg1819.dropna(how='all')
).append(seg1718.dropna(how='all')).append(seg1617.dropna(how='all')).loc[:,
['Wk','Date','Local','Score','Visitante', 'Attendance']]


laliga['Attendance']= laliga['Attendance']*1000
laliga['Torneo'] = 'Primera división'
segunda['Attendance']= segunda['Attendance']*1000
segunda['Torneo'] = 'Segunda división'


In [4]:
partidos6t = pd.concat([laliga,segunda],axis = 0)
partidos6t

,Wk,Date,Local,Score,Visitante,Attendance,Torneo
0,1.0,2021-08-13,Valencia,1–0,Getafe,9868.0,Primera división
1,1.0,2021-08-14,Cádiz,1–1,Levante,6941.0,Primera división
2,1.0,2021-08-14,Mallorca,1–1,Betis,6279.0,Primera división
3,1.0,2021-08-14,Osasuna,0–0,Espanyol,6715.0,Primera división
4,1.0,2021-08-14,Alavés,1–4,Real Madrid,3968.0,Primera división
...,...,...,...,...,...,...,...
495,42.0,2017-06-10,Valladolid,1–0,Cádiz,12812.0,Segunda división
496,42.0,2017-06-10,Gimnàstic,1–0,UCAM Murcia,11103.0,Segunda división
497,42.0,2017-06-10,Alcorcón,3–0,Lugo,4262.0,Segunda división
498,42.0,2017-06-10,Córdoba,2–1,Girona,8977.0,Segunda división


In [5]:
partidos6t['Attendance'] = partidos6t['Attendance'].fillna(0)

## Creando nuevas columnas

In [6]:
def golesvisita(marcador):
    goles = marcador.split('–')
    return int(goles[1])
def goleslocal(marcador):
    goles = marcador.split('–')
    return int(goles[0])

partidos6t['Goles Local'] = partidos6t['Score'].apply(goleslocal)
partidos6t['Goles Visitante'] = partidos6t['Score'].apply(golesvisita)
partidos6t['Total goles partido']= partidos6t['Goles Local']+partidos6t['Goles Visitante']


In [7]:
def torneo(año,mes):
    if ((año==2016) & (mes<=12)|(año==2017) & (mes<8)):
        return '16/17'
    elif ((año==2017) & (mes<=12)|(año==2018) & (mes<8)):
        return '17/18'
    elif ((año==2018) & (mes<=12)|(año==2019) & (mes<8)):
        return '18/19'
    elif ((año==2019) & (mes<=12)|(año==2020) & (mes<9)):
        return '19/20'
    elif ((año==2020) & (mes<=12)|(año==2021) & (mes<8)):
        return '20/21'
    elif ((año==2021) & (mes<=12)|(año==2022) & (mes<8)):
        return '21/22'

    
partidos6t['Año'] = partidos6t['Date'].dt.year
partidos6t['Mes'] = partidos6t['Date'].dt.month
partidos6t['Temporada'] = partidos6t.apply(lambda x: torneo(x['Año'],x['Mes']),axis = 1)


In [8]:
def resultado(string):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return 'Local'
    if int(goles[0])<int(goles[1]):
        return 'Visitante'
    else: 
        return 'Empate'
    
def resultado_equipo(string, local, visita):
    goles = string.split('–')
    if int(goles[0])>int(goles[1]):
        return local
    if int(goles[0])<int(goles[1]):
        return visita
    else: 
        return 'Empate'
    
partidos6t['Resultado'] = partidos6t['Score'].apply(resultado)
partidos6t['Ganador'] = partidos6t.apply(lambda x: resultado_equipo(x['Score'],x['Local'],x['Visitante']),axis=1)



## Tabla de posiciones hasta la fecha 33 (inclusive)

In [9]:
ej2122 = partidos6t[(partidos6t['Temporada']=='21/22')&(partidos6t['Torneo']=='Primera división')]
equip2122 = ej2122['Local'].unique().tolist()

In [10]:
def tabla_posicion(data,fecha,equipos):
    tabla = {}
    for equipo in equipos:
        data_usar = data[(data['Wk']<=fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
        puntos = 0
        for i in range(len(data_usar)):
            ganador = str(data_usar.iloc[i]['Ganador'])
            if ganador == equipo:
                puntos += 3
            if ganador == 'Empate':
                puntos +=1
        tabla[equipo] = puntos
    df = pd.DataFrame([[key, tabla[key]] for key in tabla.keys()], columns=['Equipo', 'Puntos'])
    df = df.sort_values(by='Puntos',ascending = False)
    return df

In [11]:
tabla = tabla_posicion(ej2122,33,equip2122)
tabla

,Equipo,Puntos
19,Real Madrid,78
6,Barcelona,63
7,Sevilla,63
15,Atlético Madrid,61
10,Betis,57
14,Real Sociedad,55
8,Villarreal,52
13,Athletic Club,48
3,Osasuna,44
0,Valencia,42


In [12]:
tabla_final = tabla_posicion(ej2122,38,equip2122)
tabla_final

,Equipo,Puntos
19,Real Madrid,86
6,Barcelona,73
15,Atlético Madrid,71
7,Sevilla,70
10,Betis,65
14,Real Sociedad,62
8,Villarreal,59
13,Athletic Club,55
0,Valencia,48
3,Osasuna,47


## Partidos restantes

In [13]:
def partidos_rest(data,equipo,fecha):
    rest = data[(data['Wk']>fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
    return rest

In [14]:
partidos_rest(ej2122,'Valencia',33)

,Wk,Date,Local,Score,Visitante,Attendance,Torneo,Goles Local,Goles Visitante,Total goles partido,Año,Mes,Temporada,Resultado,Ganador
373,34.0,2022-04-30,Valencia,1–1,Levante,35099.0,Primera división,1,1,2,2022,4,21/22,Empate,Empate
383,35.0,2022-05-07,Athletic Club,0–0,Valencia,41091.0,Primera división,0,0,0,2022,5,21/22,Empate,Empate
392,36.0,2022-05-10,Valencia,0–3,Betis,28091.0,Primera división,0,3,3,2022,5,21/22,Visitante,Betis
403,37.0,2022-05-14,Espanyol,1–1,Valencia,14990.0,Primera división,1,1,2,2022,5,21/22,Empate,Empate
416,38.0,2022-05-21,Valencia,2–0,Celta Vigo,16189.0,Primera división,2,0,2,2022,5,21/22,Local,Valencia


## Primera estimación

In [15]:
def primera_est(data,tabla,fecha,equipo):
    dato = data[(data['Wk']>fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
    indice = tabla[tabla['Equipo']==equipo].index.values.astype(int)[0]
    puntos = tabla.loc[indice,'Puntos']
    for i in range(len(dato)):
        if dato.iloc[i]['Local']==equipo:
            rival = dato.iloc[i]['Visitante']
            index = tabla[tabla['Equipo']==rival].index.values.astype(int)[0]
            puntos_rival = tabla.loc[index,'Puntos']
        else:
            rival = dato.iloc[i]['Local']
            index = tabla[tabla['Equipo']==rival].index.values.astype(int)[0]
            puntos_rival = tabla.loc[index,'Puntos']
        if puntos<puntos_rival:
            puntos+=3
        if puntos == puntos_rival:
            puntos +=1
    return puntos

In [16]:
tabla[tabla['Equipo']=='Valencia'] 

,Equipo,Puntos
0,Valencia,42


In [17]:
primera_est(ej2122,tabla,33,'Valencia')

48

## Segunda estimación

In [18]:
def segunda_est(data,tabla,fecha,equipo):
    dato = data[(data['Wk']>fecha)&((data['Local']==equipo)|(data['Visitante']==equipo))]
    indice = tabla[tabla['Equipo']==equipo].index.values.astype(int)[0]
    puntos = tabla.loc[indice,'Puntos']
    for i in range(len(dato)):
        if dato.iloc[i]['Local']==equipo:
            rival = dato.iloc[i]['Visitante']
            index = tabla[tabla['Equipo']==rival].index.values.astype(int)[0]
            puntos_rival = tabla.loc[index,'Puntos']
        else:
            rival = dato.iloc[i]['Local']
            index = tabla[tabla['Equipo']==rival].index.values.astype(int)[0]
            puntos_rival = tabla.loc[index,'Puntos']
        if puntos>puntos_rival:
            puntos+=3
        if puntos == puntos_rival:
            puntos +=1
    return puntos

In [19]:
tabla[tabla['Equipo']=='Valencia']

,Equipo,Puntos
0,Valencia,42


In [20]:
segunda_est(ej2122,tabla,33,'Valencia')

51

In [29]:
posiciones_33 = tabla_posicion(ej2122, 33.0, equip2122)
posiciones_33

posiciones_38 = tabla_posicion(ej2122, 38.0, equip2122)
posiciones_38

,Equipo,Puntos
19,Real Madrid,86
6,Barcelona,73
15,Atlético Madrid,71
7,Sevilla,70
10,Betis,65
14,Real Sociedad,62
8,Villarreal,59
13,Athletic Club,55
0,Valencia,48
3,Osasuna,47


In [30]:
est = primera_est(ej2122, posiciones_33, 38.0, 'Getafe')
39 - est

4

In [31]:
est = primera_est(ej2122, posiciones_33, 38.0, 'Mallorca')
39 - est

7

In [32]:
est = primera_est(ej2122, posiciones_33, 38.0, 'Granada')
38 - est

8

In [33]:
est = primera_est(ej2122, posiciones_33, 38.0, 'Espanyol')
42 - est

3